# FP16 Verification


In [11]:
import numpy as np
import random
import math


## Function definition

In [8]:
# Function to generate random FP16 binary strings
def generate_random_fp16():
    sign = random.choice(['0', '1'])
    exponent = format(random.randint(0, 31), '05b')  # 5-bit exponent
    mantissa = format(random.randint(0, 1023), '010b')  # 10-bit mantissa
    return sign + exponent + mantissa


In [52]:
class FP16AdderGoldenModel:
    def __init__(self, rounding_mode="nearest", ieee_mode="standard"):
        self.rounding_mode = rounding_mode
        self.ieee_mode = ieee_mode

    def binary_to_fp16(self, binary_str):
        sign = int(binary_str[0], 2)
        exponent = int(binary_str[1:6], 2) # exponent treated as unsigned number, so should do e-bias
        mantissa = int(binary_str[6:], 2)

        if exponent == 0 and mantissa == 0:
            return 0.0 if sign == 0 else -0.0
        elif exponent == 31:
            if mantissa == 0:
                return float('inf') if sign == 0 else float('-inf')
            else:
                if self.ieee_mode == "standard":
                    return float('nan')
                else:
                    return float('inf') if sign == 0 else float('-inf')

        exponent = exponent - 15
        if exponent < -14:
            if self.ieee_mode == "simplified":
                return +0.0 if sign == 0 else -0.0
            value = ((-1) ** sign) * (mantissa / (2 ** 10)) * (2 ** -14)
        else:
            value = ((-1) ** sign) * ((mantissa / (2 ** 10)) + 1.0) * (2 ** exponent)

        return value

    def fp16_to_binary(self, value):

        if np.isnan(value):
            return "0111111111111111"

        if np.isposinf(value):
            return "0111110000000000" # inf

        if np.isneginf(value):
            return "1111110000000000" # -inf

        if value == 0.0:
            return "1000000000000000" if math.copysign(1.0, value) == -1.0 else "0000000000000000"

        sign = 0 if value > 0 else 1
        value = abs(value)

        exponent = np.floor(np.log2(value)) + 15
        exponent = min(max(exponent, 0), 31)


        # mantissa = (value / (2 ** (exponent - 15))) - 1.0
        # mantissa_bits = int(mantissa * (2 ** 10)) & 0b1111111111

        mantissa = round((value / (2 ** (exponent - 15))) - 1.0, 10)
        # print(mantissa)
        mantissa_int = int(mantissa * (2 ** 10) + 0.5)
        # print(mantissa_int)
        if mantissa_int == 2 ** 10:
            exponent = exponent + 1 # Used to Update the exp since the mantissa may overflow under nearest rounding
        exponent_bits = int(exponent) & 0b11111
        mantissa_bits = mantissa_int & 0b1111111111
        # print(mantissa_bits)

        binary_str = f"{sign:01b}{exponent_bits:05b}{mantissa_bits:010b}"
        return binary_str

    def add(self, a_value, b_value):

        # Perform the actual addition
        if (math.isinf(a_value) and math.isinf(b_value)) and (math.copysign(1.0, a_value) != math.copysign(1.0, b_value)):
            result = float('inf')
        else:
            result = a_value + b_value

        # Manually handle overflow to the Maximum
        if math.isnan(result) or math.isinf(result):
            return float('inf') if math.copysign(1.0, result) == 1 else float('-inf')

        if result > (2 ** 16) or result < (-2 ** 16):
            return float('inf') if math.copysign(1.0, result) == 1 else float('-inf')

        if self.ieee_mode == "simplified":
            if abs(result) < (2 ** -14):
                return -0.0 if math.copysign(1.0, result) == -1 else 0.0

        return result


adder = FP16AdderGoldenModel(rounding_mode="nearest", ieee_mode="simplified")



In [60]:
def binary_to_fp16(binary_str):
    # 1. 문자열 → 정수
    uint16_val = int(binary_str, 2)

    # 2. 정수 → np.uint16
    np_uint16 = np.uint16(uint16_val)

    # 3. uint16을 float16으로 재해석
    float16_val = np.frombuffer(np_uint16.tobytes(), dtype=np.float16)[0]

    return float16_val

## Compare two function (custom vs np.float16)

In [59]:
for i in range(10000):
    # # 1. 랜덤 binary string 생성
    # a = generate_random_fp16()

    # # 2. custom 함수로 binary → float16
    # a_value = adder.binary_to_fp16(a)


    a_value = np.float16(random.random())

    # 3. numpy 기반 binary 추출
    a_float16 = np.float16(a_value)
    a_bin_numpy = format(a_float16.view(np.uint16), '016b')

    # 4. custom 함수로 float16 → binary
    a_bin_custom = adder.fp16_to_binary(a_value)

    # 5. 비교 출력
    if a_bin_custom == a_bin_numpy:
        # print(f"[{i:02}] ✅ Match: {a_bin_custom}")
        pass
    else:
        print(f"[{i:02}] ❌ Mismatch:")
        print(f"     Input binary       : {a}")
        print(f"     Float16 value       : {a_value}")
        print(f"     Custom binary       : {a_bin_custom}")
        print(f"     Numpy binary        : {a_bin_numpy}")


[3335] ❌ Mismatch:
     Input binary       : 1010010100010111
     Float16 value       : 0.0038967132568359375
     Custom binary       : 0001111111111111
     Numpy binary        : 0001101111111011
[5931] ❌ Mismatch:
     Input binary       : 1010010100010111
     Float16 value       : 0.062469482421875
     Custom binary       : 0010110000000000
     Numpy binary        : 0010101111111111
[9531] ❌ Mismatch:
     Input binary       : 1010010100010111
     Float16 value       : 5.9485435485839844e-05
     Custom binary       : 0000001111001100
     Numpy binary        : 0000001111100110


## custom binary_to_fp16 function cannot generate correct binary value

In [65]:
for i in range(10000):
    # 1. 랜덤 binary string 생성
    a = generate_random_fp16()

    # 2. custom 함수로 binary → float16
    a_value = adder.binary_to_fp16(a)


    # a_value = np.float16(random.random())

    # 3. numpy 기반 binary 추출
    # a_float16 = np.float16(a_value)
    a_float16 = binary_to_fp16(a)

    a_bin_numpy = format(a_float16.view(np.uint16), '016b')

    # 4. custom 함수로 float16 → binary
    a_bin_custom = adder.fp16_to_binary(a_value)

    # 5. 비교 출력
    if a_bin_custom == a_bin_numpy:
        # print(f"[{i:02}] ✅ Match: {a_bin_custom}")
        pass
    else:
        print(f"[{i:02}] ❌ Mismatch:")
        print(f"     Input binary       : {a}")
        print(f"     Float16 value       : {a_value}")
        print(f"     Custom binary       : {a_bin_custom}")
        print(f"     Numpy binary        : {a_bin_numpy}")


[01] ❌ Mismatch:
     Input binary       : 1111110001000010
     Float16 value       : -inf
     Custom binary       : 1111110000000000
     Numpy binary        : 1111110001000010
[14] ❌ Mismatch:
     Input binary       : 0111110000001101
     Float16 value       : inf
     Custom binary       : 0111110000000000
     Numpy binary        : 0111110000001101
[21] ❌ Mismatch:
     Input binary       : 0111111001110001
     Float16 value       : inf
     Custom binary       : 0111110000000000
     Numpy binary        : 0111111001110001
[23] ❌ Mismatch:
     Input binary       : 1000000101011010
     Float16 value       : -0.0
     Custom binary       : 1000000000000000
     Numpy binary        : 1000000101011010
[36] ❌ Mismatch:
     Input binary       : 0000000000001011
     Float16 value       : 0.0
     Custom binary       : 0000000000000000
     Numpy binary        : 0000000000001011
[56] ❌ Mismatch:
     Input binary       : 1000001101101101
     Float16 value       : -0.0
     Custom

In [ ]:
import utils as mu

x = [1.0, 2.5, 4.0, 7.0, 0.0]
y = [0.0, -1.0, -4.0, -2.0, 1.0]

print(mu.mean(x))
print(mu.var(x))
print(mu.rms(x))
print(mu.diff(x))
print(mu.xcorr(x, y))
print(mu.unwrap([0, np.pi/2, np.pi, -3*np.pi/4, -np.pi]))
print(mu.atan2(x, y))